## Preparation

We have initialized the data from Solomon's instances, and shown the scatter plot so far. It's time for us to try some solution approaches including exact algorithms and meta-heuritic algorithms. 

> Here, we are gonna try genetic algorithm first. 


In [ ]:
import os
import io
import fnmatch
from json import load, dump

import matplotlib.pyplot as plt
import pandas as pd


BASE_DIR = os.path.abspath(os.path.dirname(os.path.dirname('.')))
text_data_dir = os.path.join(BASE_DIR, 'data', 'solomon-100-original')
json_data_dir = os.path.join(BASE_DIR, 'data', 'solomon-100-json')